## Librerias

In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


## Conectar a AmigoCloud

In [2]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Datos de crédito

In [3]:
semana = 'S08'

In [4]:
query = {'query': 'select insp.canhero, prop.propiedad, lote.lote, gal.s3_filename, gal.amigocloud_url foto\
         from dataset_307560 insp\
         inner join dataset_307562 prop on insp.amigo_id=prop.insp_ref_prop\
         inner join dataset_307563 lote on prop.amigo_id=lote.prop_ref_lote\
         inner join gallery_46845 gal on lote.amigo_id=gal.source_amigo_id\
         where semana_1ra = \'{sem}\' order by canhero'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [5]:
data

[{'canhero': '14993 / VACA CUELLAR JORGE',
  'propiedad': '211 / LOS MAGUESES--VACA CUELLAR',
  'lote': 'L22',
  's3_filename': 'WhatsApp Image 2023-02-23 at 6.18.43 PM (2).jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/92779d667e674dee8d99a7fd4dac314b/eba0c6791b04453ebb03adeb42fcc21a/WhatsApp%20Image%202023-02-23%20at%206.18.43%20PM%20(2).jpeg'},
 {'canhero': '14993 / VACA CUELLAR JORGE',
  'propiedad': '211 / LOS MAGUESES--VACA CUELLAR',
  'lote': 'L14',
  's3_filename': 'IMG_20230224_112645.jpg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/3e68304ad6804965a87ecb86b47c865c/8aa66f8a856c4191b0d780a926101d91/IMG_20230224_112645.jpg'},
 {'canhero': '15162 / VELASCO ANEZ MARCO ANTONIO',
  'propiedad': '142 / SAN MARTIN--VELAZCO',
  'lote': 'L9',
  's3_filename': 'WhatsApp Image 2023-02-23 at 6.18.46 PM.jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/b4ee02e3da95485f8606fbe80344477b/605e6cfea71e436b9ac7e47bd2545f62/WhatsApp

In [6]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/template_1ra_fotos.docx")

#descargar fotos y generar lista InlineImage
lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(120)), 'canhero': foto['canhero']})
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SEMANA - ' + semana
doc.save('_' + file_name + '.docx')

WhatsApp Image 2023-02-23 at 6.18.43 PM (2).jpeg
IMG_20230224_112645.jpg
WhatsApp Image 2023-02-23 at 6.18.46 PM.jpeg
WhatsApp Image 2023-02-23 at 6.18.45 PM (1).jpeg
WhatsApp Image 2023-02-23 at 6.18.43 PM.jpeg
WhatsApp Image 2023-02-23 at 6.18.45 PM.jpeg
